In [128]:
import pandas as pd
import seaborn as sns

In [129]:
# checking the data
train_revised = pd.read_csv("Data/train_revised.csv")
train_revised.head()

,ride_id,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity
0,1442,15A,Mpesa,UZUEHCBUSO,17-10-17,7:15,Migori,Nairobi,Bus,49
1,5437,14A,Mpesa,TIHLBUSGTE,19-11-17,7:12,Migori,Nairobi,Bus,49
2,5710,8B,Mpesa,EQX8Q5G19O,26-11-17,7:05,Keroka,Nairobi,Bus,49
3,5777,19A,Mpesa,SGP18CL0ME,27-11-17,7:10,Homa Bay,Nairobi,Bus,49
4,5778,11A,Mpesa,BM97HFRGL9,27-11-17,7:12,Migori,Nairobi,Bus,49


In [130]:
# get rid of columns we don't need
train_revised.drop(columns = ["seat_number", "payment_method", "payment_receipt", "travel_to"], inplace=True)

In [131]:
# need to add column for number of tickets sold for each ride_id
number_of_ticket = train_revised.groupby("ride_id").travel_time.count()

In [132]:
# drop duplicate rows. We want each ride_id to be a unique observation.
train_revised.drop_duplicates(inplace=True)

In [133]:
# set index to ride_id
train_revised.set_index("ride_id", inplace=True)

In [134]:
# add ticket column
train_revised["number_of_ticket"] = number_of_ticket

In [135]:
train_revised.head()

,travel_date,travel_time,travel_from,car_type,max_capacity,number_of_ticket
ride_id,,,,,,
1442,17-10-17,7:15,Migori,Bus,49,1
5437,19-11-17,7:12,Migori,Bus,49,1
5710,26-11-17,7:05,Keroka,Bus,49,1
5777,27-11-17,7:10,Homa Bay,Bus,49,5
5778,27-11-17,7:12,Migori,Bus,49,31


In [136]:
# now we want to add a column with the time stuff together and make it a time series data type
train_revised["full_date"] = train_revised.travel_date + " " + train_revised.travel_time 

In [137]:
train_revised.head()

,travel_date,travel_time,travel_from,car_type,max_capacity,number_of_ticket,full_date
ride_id,,,,,,,
1442,17-10-17,7:15,Migori,Bus,49,1,17-10-17 7:15
5437,19-11-17,7:12,Migori,Bus,49,1,19-11-17 7:12
5710,26-11-17,7:05,Keroka,Bus,49,1,26-11-17 7:05
5777,27-11-17,7:10,Homa Bay,Bus,49,5,27-11-17 7:10
5778,27-11-17,7:12,Migori,Bus,49,31,27-11-17 7:12


In [138]:
# change to time series data type
train_revised.full_date = pd.to_datetime(train_revised.full_date, infer_datetime_format=True)

In [139]:
# add column with week day of ride_id
train_revised["day_of_week"] = train_revised.full_date.dt.weekday

In [140]:
# add column with month of ride_id
train_revised["month"]= train_revised.full_date.dt.month

In [141]:
# now its time to group times in morning, afternoon, evening, and night. Feel free to change around the bounds if you like.

def get_travel_time_type(hour):
    return (
        "morning" if 5 <= hour <= 11
        else
        "afternoon" if 12 <= hour <= 17
        else
        "evening" if 18 <= hour <= 22
        else
        "night"
    )
train_revised['travel_time_type'] = train_revised.full_date.apply(lambda x: get_travel_time_type(x.hour)) 

In [142]:
train_revised.head()

,travel_date,travel_time,travel_from,car_type,max_capacity,number_of_ticket,full_date,day_of_week,month,travel_time_type
ride_id,,,,,,,,,,
1442,17-10-17,7:15,Migori,Bus,49,1,2017-10-17 07:15:00,1,10,morning
5437,19-11-17,7:12,Migori,Bus,49,1,2017-11-19 07:12:00,6,11,morning
5710,26-11-17,7:05,Keroka,Bus,49,1,2017-11-26 07:05:00,6,11,morning
5777,27-11-17,7:10,Homa Bay,Bus,49,5,2017-11-27 07:10:00,0,11,morning
5778,27-11-17,7:12,Migori,Bus,49,31,2017-11-27 07:12:00,0,11,morning


In [143]:
# getting rid of columns we don't need
#If you think we need the travel_time and travel_date columns seperately feel free to edit this
train_revised.drop(columns = ["travel_time", "travel_date", "max_capacity"], inplace=True)

In [144]:
# changing categorical columns to category type
train_revised.car_type = train_revised.car_type.astype("category")
train_revised.travel_from = train_revised.travel_from.astype("category")
train_revised.travel_time_type = train_revised.travel_time_type.astype("category")
train_revised.day_of_week = train_revised.day_of_week.astype("category")
train_revised.month = train_revised.month.astype("category")

In [145]:
# Shifting the number_of_ticket column to the end to make it look nicer
df1 = train_revised.pop('number_of_ticket')
train_revised['number_of_ticket'] = df1

In [146]:
train_revised.head()

,travel_from,car_type,full_date,day_of_week,month,travel_time_type,number_of_ticket
ride_id,,,,,,,
1442,Migori,Bus,2017-10-17 07:15:00,1,10,morning,1
5437,Migori,Bus,2017-11-19 07:12:00,6,11,morning,1
5710,Keroka,Bus,2017-11-26 07:05:00,6,11,morning,1
5777,Homa Bay,Bus,2017-11-27 07:10:00,0,11,morning,5
5778,Migori,Bus,2017-11-27 07:12:00,0,11,morning,31


In [147]:
#Now its data analysis time! Although we could add some more columns or something if you guys want.